In [52]:
import pandas as pd
#import plotly.express as px
import numpy as np
#import geopandas as gpd
import os
#import pm4py

# machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.tree import plot_tree
# nice plots & tables
#import plotly.express as px
#import plotly.graph_objects as go
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
# to make a counter object
from collections import Counter
from sklearn.model_selection import train_test_split

In [53]:
df = pd.read_csv("burglary_data_city.csv")
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Longitude,Latitude,month,year,median house price,burglarys city,burglarys lsoa,city
0,0,29126.0,29126.0,-0.252285,50.834953,12.0,10.0,290500.0,33.0,2.0,Adur
1,1,2913.5,2913.5,-3.440349,54.698378,12.0,10.0,106500.0,34.0,2.5,Allerdale
2,2,3077.0,3077.0,-1.388190,53.027519,12.0,10.0,157500.0,57.0,2.0,Amber Valley
3,3,29173.5,29173.5,-0.635302,50.807424,12.0,10.0,265500.0,62.0,2.0,Arun
4,4,25441.5,25441.5,-1.256562,53.105927,12.0,10.0,126500.0,78.0,2.0,Ashfield
...,...,...,...,...,...,...,...,...,...,...,...
45185,45185,4320862.0,4320862.0,-2.078257,52.115438,10.0,21.0,275000.0,57.0,2.0,Wychavon
45186,45186,4320310.0,4320310.0,-0.764648,51.632315,10.0,21.0,372500.0,53.0,2.0,Wycombe
45187,45187,4309338.0,4309338.0,-3.005697,53.879259,10.0,21.0,190000.0,25.0,1.0,Wyre
45188,45188,4320907.0,4320907.0,-2.266527,52.369114,10.0,21.0,180000.0,33.0,1.0,Wyre Forest


In [54]:
def balance_dataset(df, max_class, column):
    unique = list(df[column].unique())
    df_nf = df[df[column]==unique[0]].sample(frac=1)[:max_class]
    df_arr = df[df[column]==unique[1]].sample(frac=1)[:max_class]
    df_cr = df[df[column]==unique[2]].sample(frac=1)[:max_class]
    df_war = df[df[column]==unique[3]].sample(frac=1)[:max_class]
    df_sum = df[df[column]==unique[4]].sample(frac=1)[:max_class]
    df_fine = df[df[column]==unique[5]].sample(frac=1)[:max_class]
    df_all = pd.concat([df_nf,df_arr,df_cr,df_war,df_sum,df_fine])
    df_all = df_all.sample(frac=1)
    df_all.reset_index(inplace=True, drop=True)
    return df_all

In [55]:
df_all = balance_dataset(df, 20000, 'burglarys city')
df_all

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Longitude,Latitude,month,year,median house price,burglarys city,burglarys lsoa,city
0,20784,2191340.5,2191340.5,1.156383,51.876553,11.0,15.0,200000.0,78.0,3.0,Tendring
1,34479,3594723.0,3594723.0,-1.094581,52.986241,3.0,19.0,211500.0,39.0,1.0,Gedling
2,11306,1274777.0,1274777.0,-0.480377,53.180009,8.0,13.0,215000.0,39.0,2.0,North Kesteven
3,24246,2538108.5,2538108.5,-0.733069,51.331121,9.0,16.0,360750.0,34.0,1.0,Surrey Heath
4,37468,3851420.0,3851420.0,-2.485278,53.739720,12.0,19.0,92000.0,57.0,2.0,Blackburn with Darwen
...,...,...,...,...,...,...,...,...,...,...,...
2582,16497,1815749.5,1815749.5,-3.369079,51.752272,11.0,14.0,95750.0,33.0,2.0,Merthyr Tydfil
2583,36350,3727163.0,3727163.0,-2.755856,50.953076,8.0,19.0,235000.0,57.0,2.0,South Somerset
2584,17125,1862580.5,1862580.5,0.091832,52.566022,1.0,15.0,186750.0,62.0,2.0,Fenland
2585,39939,4026345.0,4026345.0,-4.134270,52.926729,7.0,20.0,153750.0,33.0,2.0,Gwynedd


In [56]:
X = df_all[['month', 'year', 'median house price', 'Latitude', 'Longitude']]
Y = df_all['burglarys city']

X = pd.get_dummies(X)


In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [58]:
y_test

544     34.0
2499    57.0
554     62.0
422     62.0
798     33.0
        ... 
1386    57.0
879     39.0
1857    62.0
2280    33.0
1780    62.0
Name: burglarys city, Length: 259, dtype: float64

In [62]:
clf = RandomForestRegressor(max_depth=15, random_state=0, bootstrap=True, n_estimators=5000)
model = clf.fit(X_train, y_train)
predictions = model.predict(X_test)
print("MAE:", metrics.mean_absolute_error(y_test, predictions), "R2", metrics.r2_score(y_test, predictions), "MSE", metrics.mean_squared_error(y_test, predictions))

MAE: 9.95548482716615 R2 0.30278186016989683 MSE 148.2562382556023


In [1]:

predictions

NameError: name 'predictions' is not defined